# MNIST pixel rnn

In [10]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.datasets import mnist
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, Dense, Activation
from keras.layers import LSTM,SimpleRNN
from keras.optimizers import Adam

In [11]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

In [5]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [6]:
x_train = x_train.reshape(x_train.shape[0], -1, 1)
x_test = x_test.reshape(x_test.shape[0], -1, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
#training data:60000,28*28=784,1是0~1中間的一個數字

x_train shape: (60000, 784, 1)
60000 train samples
10000 test samples


In [7]:
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [8]:
model = Sequential()
model.add(SimpleRNN(100,
                    activation='relu',
                    input_shape=x_train.shape[1:]))
model.add(Dense(10))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-6),
              metrics=['accuracy'])

#crossentropy,mse都可以

In [ ]:
model.fit(x_train, y_train,
          batch_size=32,
          epochs=20,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
16448/60000 [=======>......................] - ETA: 475s - loss: 2.3026 - acc: 0.1097

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=0)
print('IRNN test score:', scores[0])
print('IRNN test accuracy:', scores[1])

# imdb影評分析

In [1]:
import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

Using TensorFlow backend.


In [2]:
#引入套件
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb #電影資料庫

In [3]:
#引入資料
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=20000)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


In [4]:
#將資料整理成方形的array
x_train = sequence.pad_sequences(x_train, maxlen=80)
x_test = sequence.pad_sequences(x_test, maxlen=80)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [5]:
#建立模型
model = Sequential()
model.add(Embedding(20000, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))#dropout是有時候訓練有時候不訓練
model.add(Dense(1, activation='sigmoid'))

In [6]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [7]:
x_train[0]

array([   15,   256,     4,     2,     7,  3766,     5,   723,    36,
          71,    43,   530,   476,    26,   400,   317,    46,     7,
           4, 12118,  1029,    13,   104,    88,     4,   381,    15,
         297,    98,    32,  2071,    56,    26,   141,     6,   194,
        7486,    18,     4,   226,    22,    21,   134,   476,    26,
         480,     5,   144,    30,  5535,    18,    51,    36,    28,
         224,    92,    25,   104,     4,   226,    65,    16,    38,
        1334,    88,    12,    16,   283,     5,    16,  4472,   113,
         103,    32,    15,    16,  5345,    19,   178,    32], dtype=int32)

In [8]:
y_train[0]

1

In [9]:
#開始訓練#epochs:次數
model.fit(x_train, y_train,
          batch_size=32,
          epochs=2,
          validation_data=(x_test, y_test))


Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 430s - loss: 0.4595 - acc: 0.7821 - val_loss: 0.4331 - val_acc: 0.8097
Epoch 2/2
25000/25000 [==============================] - 414s - loss: 0.2992 - acc: 0.8789 - val_loss: 0.3749 - val_acc: 0.8306


In [10]:
#看看成果
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

NameError: name 'batch_size' is not defined

# 讀入套件與資料

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation, Input, Flatten, LeakyReLU, SimpleRNN, GRU, LSTM, Embedding
from keras.optimizers import SGD
import pandas as pd


#讀入資料(這裡基本上跟前天一樣)
DIRPATH = 'etf_data/' #ETF資料夾位置
df = pd.read_csv(DIRPATH+'AADR.csv')
df.head()

In [ ]:
#這裡確認一下形狀
df.shape

## 看起來共有1656天的資料呢

# 於是來整理資料（這裡做法跟前天一樣）

In [ ]:
Open = np.array(list(reversed(df['Open'])))

#整理資料成為我們要的形狀（一樣是只放開盤價作為Data）
X = []
Y = []
for index in range(Open.shape[0]-20):
    X.append(Open[index:index+20])
    Y.append(Open[index+20])
x = np.array(X)
y = np.array(Y)

x = x.reshape(x.shape[0], 20, 1)
y = y.reshape(y.shape[0],1)

#分成訓練及與測試集, 前1300組作為訓練，後335組作為測試(這裡為了配合input要求的形狀動了點手腳)

x_train = x[0:1300]
y_train = y[0:1300]
x_test = x[1300:1636]
y_test  = y[1300:1636]



In [ ]:
x.shape

In [ ]:
Open.shape

In [ ]:
x_train.shape

# 一般的RNN

In [ ]:
model = Sequential()
model.add(SimpleRNN(128, dropout=0.2, recurrent_dropout=0.2, input_shape = (20, 1)))#這裡規定要放二維以上
model.add(Dense(1))
model.summary()
model.compile(loss='mse', optimizer='SGD')

model.fit(x_train, y_train)


In [ ]:
#使用模型
result = model.predict_classes(x_test)

In [ ]:
#看看結果(當然不會準，資料放太少)
result 

# GRU

In [ ]:
model = Sequential()
model.add(GRU(128, dropout=0.2, recurrent_dropout=0.2, input_shape=(20,1)))
model.add(Dense(1))
model.add(LeakyReLU())
model.summary()
model.compile(loss='mse', optimizer='SGD')

model.fit(x_train, y_train)


In [ ]:
result = model.predict_classes(x_test)
result

# LSTM

In [ ]:
model = Sequential()
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, input_shape=(20, 1)))
model.add(Dense(1))
model.summary()
model.compile(loss='mse', optimizer='SGD', metrics=['accuracy'])
model.fit(x_train, y_train)



# 實務上，我們會用ramdom來tune參數

In [ ]:
num_rnn = np.random.randint(175, 275)
rate_drop_rnn = 0.15 + np.random.rand() * 0.25
rate_drop         = 0.15 + np.random.rand() * 0.25

In [ ]:
model = Sequential()
model.add(LSTM(num_rnn, dropout=rate_drop, recurrent_dropout= rate_drop_rnn, input_shape=(20,1)))
model.add(Dense(1))
model.add(LeakyReLU())
model.summary()
model.compile(loss='mse', optimizer='SGD', metrics=['accuracy'])
model.fit(x_train, y_train)
score = model.evaluate(x_test, y_test)
print(' ')
print('LSTM')
print(score)